In [ ]:
import numpy as np
from PIL import Image
from scipy.fftpack import idct
import pickle

class CustomJPEGDecoder:
    def __init__(self):
        # We’ll assign the same standard quantization matrices here
        self._lumQuant = np.array([
            [16, 11, 10, 16, 24, 40, 51, 61],
            [12, 12, 14, 19, 26, 58, 60, 55],
            [14, 13, 16, 24, 40, 57, 69, 56],
            [14, 17, 22, 29, 51, 87, 80, 62],
            [18, 22, 37, 56, 68, 109, 103, 77],
            [24, 35, 55, 64, 81, 104, 113, 92],
            [49, 64, 78, 87, 103, 121, 120, 101],
            [72, 92, 95, 98, 112, 100, 103, 99]
        ])
        self._chrQuant = np.array([
            [17, 18, 24, 47, 99, 99, 99, 99],
            [18, 21, 26, 66, 99, 99, 99, 99],
            [24, 26, 56, 99, 99, 99, 99, 99],
            [47, 66, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99]
        ])
        self._zigzagMap = self._computeZigzag(8)

    def _computeZigzag(self, n):
        idx = np.empty((n*n, 2), dtype=int)
        k = 0
        for s in range(0, 2*n - 1):
            if s % 2 == 0:
                x = min(s, n - 1)
                y = s - x
                while x >= 0 and y < n:
                    idx[k] = [x, y]
                    k += 1
                    x -= 1
                    y += 1
            else:
                y = min(s, n - 1)
                x = s - y
                while y >= 0 and x < n:
                    idx[k] = [x, y]
                    k += 1
                    x += 1
                    y -= 1
        return idx

    def _inverseZigzag(self, arr):
        block = np.zeros((8, 8))
        for i, (x, y) in enumerate(self._zigzagMap):
            block[x, y] = arr[i]
        return block

    def _runLengthDecode(self, encoded):
        out = []
        for count, value in encoded:
            if (count, value) == (0, 0):
                break
            out.extend([0] * count)
            out.append(value)
        while len(out) < 64:
            out.append(0)
        return np.array(out)

    def _dequantize(self, block, Q):
        return block * Q

    def _applyIDCT(self, block):
        return idct(idct(block.T, norm='ortho').T, norm='ortho')

    def _mergeBlocks(self, blocks, dims, blkSize=8):
        h, w = dims
        merged = np.zeros((h, w))
        k = 0
        for i in range(0, h, blkSize):
            for j in range(0, w, blkSize):
                merged[i:i+blkSize, j:j+blkSize] = blocks[k]
                k += 1
        return merged

    def _upsample(self, channel, target):
        upsampled = np.repeat(np.repeat(channel, 2, axis=0), 2, axis=1)
        return upsampled[:target[0], :target[1]]

    def _ycbcr2rgb(self, Y, Cb, Cr):
        R = Y + 1.402 * (Cr - 128)
        G = Y - 0.344136 * (Cb - 128) - 0.714136 * (Cr - 128)
        B = Y + 1.772 * (Cb - 128)
        rgb = np.stack((R, G, B), axis=-1)
        return np.clip(rgb, 0, 255).astype(np.uint8)

    def decode(self, compFile, outputPNG="output_image.png"):
        with open(compFile, "rb") as fp:
            data = pickle.load(fp)
        quality = data['quality']
        # Recompute scaled quantization matrices based on quality
        if quality < 50:
            factor = 5000 / quality
        else:
            factor = 200 - 2 * quality
        Qlum = np.floor((self._lumQuant * factor + 50) / 100)
        Qlum[Qlum == 0] = 1
        Qchr = np.floor((self._chrQuant * factor + 50) / 100)
        Qchr[Qchr == 0] = 1

        def decodeChannel(encodedBlocks, Q, shape):
            recovered = []
            for enc in encodedBlocks:
                arr = self._runLengthDecode(enc)
                block = self._inverseZigzag(arr)
                dequant = self._dequantize(block, Q)
                spatial = self._applyIDCT(dequant) + 128
                spatial = np.clip(spatial, 0, 255)
                recovered.append(spatial)
            return self._mergeBlocks(recovered, shape)

        Y_rec = decodeChannel(data['Y'], Qlum, data['Y_shape'])
        Cb_rec = decodeChannel(data['Cb'], Qchr, data['Cb_shape'])
        Cr_rec = decodeChannel(data['Cr'], Qchr, data['Cr_shape'])
        # Upsample chroma channels to original image size
        orig_h, orig_w, _ = data['orig_shape']
        Cb_up = self._upsample(Cb_rec, (orig_h, orig_w))
        Cr_up = self._upsample(Cr_rec, (orig_h, orig_w))
        rgb = self._ycbcr2rgb(Y_rec, Cb_up, Cr_up)
        Image.fromarray(rgb).save(outputPNG, "PNG")
        print("Decoding complete. JPEG saved as", outputPNG)

def main():
    decoder = CustomJPEGDecoder()
    decoder.decode("compressed.bin", "output_image.png")

if __name__ == "__main__":
    main()


Decoding complete. JPEG saved as output_image.jpg
